## Import library

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib as plt
import pandas as pd
#for AOI and terrain data
from helper_module import get_aoi_from_gaul
#image collection
from Modul_1 import ReflectanceData
#sample separability and analysis
from Modul_4 import sample_quality
#spectral transformation (optional)
from Modul_5 import spectral_transformation_calcultator
from Modul_5 import spectral_transformation_calcultator, indexcategory
#feature extraction, hyperparameter tuning, and classification 
from Modul_61 import FeatureExtraction
from Modul_62 import Hyperparameter_tuning
from Modul_63and7 import Generate_and_evaluate_LULC
ee.Initialize()
ee.Authenticate()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


## Get Image Collection

In [ ]:
# --- Area of Interest (AOI) ---
# Set the country and province for the AOI using GAUL admin boundaries
COUNTRY = "Indonesia"
PROVINCE = "Sumatera Selatan"
aoi = get_aoi_from_gaul(country=COUNTRY, province=PROVINCE)
#Intialize the relfectance class data function
optical_reflectance = ReflectanceData()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2018-12-31'
#get the image collection and corresponding statistics
landsat_data, stats = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR', 
                                                           cloud_cover=40, compute_detailed_stats=True)
#visualization parameter
l8_sr_visparam = {
    'min': 0,
    'max': 0.4,
    'gamma': [0.95, 1.1, 1],
    'bands':['NIR', 'RED', 'GREEN']
}
#create mosaic between image collection, sort them based on the cloud cover property, and clip based on AOI
mosaic_landsat = landsat_data.sort('CLOUD_COVER_LAND').mosaic().clip(aoi)
#Add the data to the map
Map = geemap.Map()
Map.addLayer(mosaic_landsat, l8_sr_visparam, 'L8 SR Mosaic')
Map 
# set center of the map in the area of interest
Map.centerObject(aoi, 7)
# adding the data
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')

2025-09-18 15:33:56,781 - ReflectanceData - INFO - ReflectanceData initialized.


2025-09-18 15:33:58,510 - ReflectanceData - INFO - Starting data fetch for Landsat 8 Surface Reflectance
2025-09-18 15:33:58,511 - ReflectanceData - INFO - Date range: 2017-01-01 to 2018-12-31
2025-09-18 15:33:58,512 - ReflectanceData - INFO - Cloud cover threshold: 40%
2025-09-18 15:34:24,402 - ReflectanceData - INFO - Initial collection (before cloud filtering): 409 images
2025-09-18 15:34:24,404 - ReflectanceData - INFO - Date range of available images: 2017-01-06 to 2018-12-27
2025-09-18 15:34:33,667 - ReflectanceData - INFO - After cloud filtering (<40%): 113 images
2025-09-18 15:34:33,668 - ReflectanceData - INFO - Cloud cover of selected images: 2.9% - 40.0%
2025-09-18 15:34:33,669 - ReflectanceData - INFO - Average cloud cover: 26.2%
2025-09-18 15:34:33,670 - ReflectanceData - INFO - Images span from 2017-01-13 to 2018-12-18
2025-09-18 15:34:33,671 - ReflectanceData - INFO - Path/Row tiles: 123/62, 123/63, 124/61, 124/62, 124/63, 124/64, 125/61, 125/62, 125/63, 126/62


In [ ]:
#Hitung cloud cover yang ada di AOI
#What happened if the cloud did not satify the needs?
#Visual good to go, optional buat download citra()
#feedback: Rubah persentase awan (1) mengganti tahun perekaman (2)
#provide detail information regarding mosaicking process.
#valid pixel (use ee reducer count) map
Map

Map(bottom=17018.0, center=[-3.6669277409287235, 104.12840724623969], controls=(WidgetControl(options=['positi…

## Applied Preprocessing

In [3]:
from optical_preprocessing import apply_brdf, terrain_correction
brdf_landsat = landsat_data.map(apply_brdf)
terrain_corrected = terrain_correction(brdf_landsat)
Map.addLayer(brdf_landsat, l8_sr_visparam, 'BRDF_Cor')
Map.addLayer(terrain_corrected, l8_sr_visparam, 'TerrainBRDF_Corrected')

## Perform Temporal Compositing on Image Collection

In [4]:
temporal_comp_landsat, details = optical_reflectance.temporal_compositing(aoi, collection=landsat_data, data_type='ms')
#median + stdDev + percentiles
print(details['Other bands filtered'].bandNames().getInfo())
#Blue band median
print(details['Blue band filtered'].bandNames().getInfo())
#Map.addLayer(valid_pixel, {}, 'Valid pixel')
#Map

2025-09-18 13:50:56,484 - ReflectanceData - INFO - Available bands: ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']
2025-09-18 13:50:59,024 - ReflectanceData - INFO - Final multispectral composite created with 16 bands


['GREEN_median', 'GREEN_variance', 'GREEN_mean', 'RED_median', 'RED_variance', 'RED_mean', 'NIR_median', 'NIR_variance', 'NIR_mean', 'SWIR1_median', 'SWIR1_variance', 'SWIR1_mean', 'SWIR2_median', 'SWIR2_variance', 'SWIR2_mean']
['BLUE_median']


## Perform Spectral Transformation Calculation and Temporal Compositing

In [ ]:
spectral_transform = spectral_transformation_calcultator()
spectral_transform.list_category()
spectral_transform.indices_list(None)
#index_collection = proc_landsat.map(
  #  lambda img: spectral_transform.calculate_index(img, index=['NDVI']))

2025-09-18 14:35:59,922 - root - INFO - Spectral transformation category:
2025-09-18 14:35:59,923 - root - INFO -   - vegetation: 9 indices
2025-09-18 14:35:59,923 - root - INFO -   - moisture: 2 indices
2025-09-18 14:35:59,924 - root - INFO -   - burn: 3 indices
2025-09-18 14:35:59,925 - root - INFO -   - water: 2 indices
2025-09-18 14:35:59,925 - root - INFO -   - soil_buildup: 3 indices
2025-09-18 14:35:59,926 - root - INFO -   - tasseleed_cap: 1 indices
2025-09-18 14:35:59,927 - root - INFO - Supported spectral transformation:
2025-09-18 14:35:59,928 - root - INFO -  - NDVI: Normalized Difference Vegetation Index(Reference: https://doi.org/10.1016/0034-4257(79)90013-0)
2025-09-18 14:35:59,929 - root - INFO -  - GNDVI: Green Normalized Difference Vegetation Index(Reference: https://doi.org/10.1016/S0034-4257(96)00072-7 )
2025-09-18 14:35:59,929 - root - INFO -  - MSAVI: Modified Soil Adjusted Vegetation Index(Reference: https://doi.org/10.1016/0034-4257(94)90134-1)
2025-09-18 14:35:

In [38]:
nd_collection = terrain_corrected.map(
    lambda img: spectral_transform.calculate_index(img, index=['NDVI', 'GNDVI', 'GBNDVI'])
)
tascap_collection = terrain_corrected.map(
    lambda img: spectral_transform.calculate_index(img, categories=[indexcategory.tascap])
)

index_composite, detail = optical_reflectance.temporal_compositing(aoi, collection=nd_collection, data_type='idx')
tascap_composite, info = optical_reflectance.temporal_compositing(aoi, collection=tascap_collection, data_type='idx')
#Check on one data
cvi = index_composite.select('CVI_median')
v_map = geemap.Map()
#v_map.addLayer(cvi, {'min': 1, 'max': 10, 'palette':  ['red', 'yellow', 'green']}, 'CVI_Median')
#v_map.centerObject(aoi, 8)
#v_map
#veg_index = spectral_transform.calculate_index(median_landsat,  categories=[indexcategory.vegetation, indexcategory.moisture])

2025-09-18 14:51:57,627 - root - INFO - Starting spectral indices calculation
2025-09-18 14:51:57,628 - root - INFO - Calculating 3 indices: ['NDVI', 'GNDVI', 'GBNDVI']
2025-09-18 14:51:57,630 - root - INFO - Successfully calculated NDVI: Normalized Difference Vegetation Index
2025-09-18 14:51:57,631 - root - INFO - Successfully calculated GNDVI: Green Normalized Difference Vegetation Index
2025-09-18 14:51:57,632 - root - INFO - Successfully calculated GBNDVI: Green-Blue NDVI
2025-09-18 14:51:57,633 - root - INFO - Calculatation sucessfull on 3 spectral indices
2025-09-18 14:51:57,634 - root - INFO - Starting spectral indices calculation
2025-09-18 14:51:57,635 - root - INFO - Calculating 3 indices: ['NDVI', 'GNDVI', 'GBNDVI']
2025-09-18 14:51:57,636 - root - INFO - Successfully calculated NDVI: Normalized Difference Vegetation Index
2025-09-18 14:51:57,637 - root - INFO - Successfully calculated GNDVI: Green Normalized Difference Vegetation Index
2025-09-18 14:51:57,638 - root - INFO

## Stack all of the composite

In [39]:
#stack all composite
optical_composite = ee.Image.cat([temporal_comp_landsat, index_composite, tascap_composite])
#Inspect how many bands
print(f"final covariates stack size: {optical_composite.bandNames().size().getInfo()}")

final covariates stack size: 22


## Perform Sample Quality Analysis

In [ ]:
#Import the sample quality functions
from sample_analysis import sample_quality
#Defined the Region of Interest
#1. Upload/tulis skema klasifikasi
#2. upload shapefile TD
#3. matching antara shapefile dan skema klasifikasi
#4. Visualisasi sampel 
#5. FIlter sampel (wajib di dalam AOI) jika diluar, akan ada warning sebarapa banyak sampel diluar AOI dan diabaikan
# use geemap.shp_to_ee to used local shapefile (without upload to asset)
labeled_roi = ee.FeatureCollection('projects/ee-agilakbar/assets/Reference_data_sumsel_test')
#Conduct the analysis
analyzer = sample_quality(training_data=labeled_roi, 
    image= mosaic_landsat, 
    class_property='LULC_ID',           # Column with numeric IDs (1, 2, 3, etc.)
    region= aoi,
    class_name_property='New_LCID'          # Column with names ('Forest', 'Urban', 'Water', etc.)
)
# Extract spectral values
pixel_extract = analyzer.extract_spectral_values(scale=30, max_pixels_per_class=5000)
samples_statistic = analyzer.sample_stats()
sample_df = analyzer.get_sample_stats_df()
display(sample_df)
#user interface via streamlit create a page a single page web. 

Extracted spectral values for 2703 samples across 19 classes


,LULC_ID,New_LCID,Sample_Count,Proportion,Percentage
0,1.0,Acacia plantation,53,0.0196,1.96
1,10.0,Mixed Garden,205,0.0758,7.58
2,11.0,Oil palm monoculture,402,0.1487,14.87
3,12.0,Other Crops,21,0.0078,0.78
4,13.0,Rice Field,114,0.0422,4.22
5,14.0,Rubber agroforest,37,0.0137,1.37
6,15.0,Rubber monoculture,1212,0.4484,44.84
7,16.0,Settlement,324,0.1199,11.99
8,17.0,Shrub,15,0.0055,0.55
9,18.0,Tea plantation,7,0.0026,0.26


In [ ]:
#get pixel values statistic
pixel_stats = analyzer.sample_pixel_stats(pixel_extract)
pixel_stats_df = analyzer.get_sample_pixel_stats_df(pixel_extract)
display(pixel_stats_df)
#Scattergram, long table, summary

,LULC_ID,New_LCID,Band,Mean,Std,Min,Max,Median,Count
0,1,Acacia plantation,BLUE,0.04,0.03,-0.03,0.16,0.03,53
1,1,Acacia plantation,GREEN,0.08,0.03,0.03,0.21,0.07,53
2,1,Acacia plantation,NIR,0.32,0.06,0.21,0.43,0.32,53
3,1,Acacia plantation,RED,0.07,0.03,0.01,0.20,0.07,53
4,1,Acacia plantation,SWIR1,0.21,0.06,0.05,0.33,0.20,53
...,...,...,...,...,...,...,...,...,...
109,19,Water Body,GREEN,0.08,0.02,0.03,0.15,0.07,65
110,19,Water Body,NIR,0.21,0.08,0.06,0.46,0.21,65
111,19,Water Body,RED,0.07,0.03,0.02,0.17,0.07,65
112,19,Water Body,SWIR1,0.13,0.06,0.02,0.32,0.13,65


In [9]:
#conduct separability analysis
separability_analysis = analyzer.get_separability_df(pixel_extract)
display(separability_analysis)

,Class1_ID,Class1_Name,Class2_ID,Class2_Name,JM_Distance,Separability_Level
0,15,Rubber monoculture,11,Oil palm monoculture,0.221,Class confusions
1,11,Oil palm monoculture,15,Rubber monoculture,0.221,Class confusions
2,14,Rubber agroforest,15,Rubber monoculture,0.385,Class confusions
3,15,Rubber monoculture,14,Rubber agroforest,0.385,Class confusions
4,11,Oil palm monoculture,1,Acacia plantation,0.396,Class confusions
...,...,...,...,...,...,...
337,2,Cane,4,Coconut monoculture,1.956,Good Separability
338,18,Tea plantation,4,Coconut monoculture,1.972,Good Separability
339,4,Coconut monoculture,18,Tea plantation,1.972,Good Separability
340,4,Coconut monoculture,7,Logged over forest-high density,1.981,Good Separability


In [10]:
lowest_sep = analyzer.lowest_separability(pixel_extract)
display(lowest_sep)

,Class1_ID,Class1_Name,Class2_ID,Class2_Name,JM_Distance,Separability_Level,Interpretation
0,15,Rubber monoculture,11,Oil palm monoculture,0.221,Class confusions,Class confusions
1,11,Oil palm monoculture,15,Rubber monoculture,0.221,Class confusions,Class confusions
2,14,Rubber agroforest,15,Rubber monoculture,0.385,Class confusions,Class confusions
3,15,Rubber monoculture,14,Rubber agroforest,0.385,Class confusions,Class confusions
4,11,Oil palm monoculture,1,Acacia plantation,0.396,Class confusions,Class confusions
5,1,Acacia plantation,11,Oil palm monoculture,0.396,Class confusions,Class confusions
6,13,Rice Field,19,Water Body,0.450,Class confusions,Class confusions
7,19,Water Body,13,Rice Field,0.450,Class confusions,Class confusions
8,10,Mixed Garden,15,Rubber monoculture,0.499,Class confusions,Class confusions
9,15,Rubber monoculture,10,Mixed Garden,0.499,Class confusions,Class confusions


In [11]:
# Overall separability summary
sep_summary = analyzer.sum_separability(pixel_extract)
print("Overall Separability Statistics:")
display(sep_summary)

Overall Separability Statistics:


,Total Pairs,Good Separability Pairs,Weak Separability Pairs,Worst Separability Pairs
0,342,50,180,112


with temporal composite:
Good Separability Pairs	
142
Weak Separability Pairs	
186
Worst Separability Pairs
14

Multispectral Bands Only:
Good Separability Pairs	
50
Weak Separability Pairs	
180
Worst Separability Pairs
112



## Feature Extraction

In [ ]:
features = FeatureExtraction()
random_train, random_test = features.random_split(mosaic_landsat, labeled_roi, class_property='LULC_ID', split_ratio=0.7)
#print('Training points:', random_train.size().getInfo())
#print('Validation points:', random_test.size().getInfo())
strafied_train, stratified_test = features.stratified_split(labeled_roi, mosaic_landsat, class_prop='LULC_ID', train_ratio=0.5)
#print('Straified_train', strafied_train.size().getInfo())
#print('Straified test', stratified_test.size().getInfo())

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


## Hyperparameter Tuning

In [ ]:
tuning_option = Hyperparameter_tuning()
num_trees = [50, 100, 150]
var_split = [5, 8, 15, 21,25]
min_leaf_pop = [1,5,9]
simple_tuning = tuning_option.Hard_classification_tuning(strafied_train, stratified_test, image=mosaic_landsat, class_property='LULC_ID',
                                                         n_tree_list=num_trees, var_split_list=var_split, min_leaf_pop_list=min_leaf_pop)

## Generate LULC

In [ ]:
generate_eval_lulc = Generate_and_evaluate_LULC()
#Multiclass hard classification
multiclass = generate_eval_lulc.multiclass_classification(strafied_train, class_property='LULC_ID', image=mosaic_landsat,
                                                          ntrees=300, min_leaf=2)
#Probability based classification/soft classification
soft_classification = generate_eval_lulc.ovr_classification(training_data=strafied_train, class_property='LULC_ID', image=mosaic_landsat,
                                                            include_final_map=True, ntrees=300, min_leaf=1)
